In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess/dataset"
import os
import sys
file_path = "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess"
os.chdir(file_path)
__package__ == "trainer"

Mounted at /content/drive
dpo_test.jsonl	 __init__.py	   lm_dataset.py      __pycache__
dpo_train.jsonl  lm_dataset.ipynb  pretrain_hq.jsonl  sft_mini_512.jsonl


False

In [2]:
'''
from datasets import load_dataset

# 1. 加载数据集
ds = load_dataset("Magpie-Align/MagpieLM-DPO-Data-v0.1")

# 2. 选择一个拆分
train_ds = ds['train']

# 3. 将数据集转换为 JSONL 文件并保存
train_ds.to_json('magpie_dpo_train.jsonl', lines=True)


test_ds = ds['test']
test_ds.to_json('magpie_dpo_test.jsonl', lines=True)

print("数据集已成功转换为 JSONL 文件并保存到本地。")
'''

'\nfrom datasets import load_dataset\n\n# 1. 加载数据集\nds = load_dataset("Magpie-Align/MagpieLM-DPO-Data-v0.1")\n\n# 2. 选择一个拆分\ntrain_ds = ds[\'train\']\n\n# 3. 将数据集转换为 JSONL 文件并保存\ntrain_ds.to_json(\'magpie_dpo_train.jsonl\', lines=True)\n\n\ntest_ds = ds[\'test\']\ntest_ds.to_json(\'magpie_dpo_test.jsonl\', lines=True)\n\nprint("数据集已成功转换为 JSONL 文件并保存到本地。")\n'

In [3]:
import time
import math
import warnings
import torch
import torch.nn.functional as F
from contextlib import nullcontext
from torch import optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
from dataset.lm_dataset import DPODataset
from datasets import load_dataset
warnings.filterwarnings('ignore')


def Logger(content):
    print(content)


def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))


def logits_to_probs(logits, labels):
    log_probs = F.log_softmax(logits, dim=2)
    probs = torch.gather(log_probs, dim=2, index=labels.unsqueeze(2)).squeeze(-1)
    return probs


def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/')
    model = MiniMindForCausalLM(lm_config)
    # sft_path = "./out/full_sft_512.pth"
    sft_path = "./out/rlhf_512.pth"
    state_dict = torch.load(sft_path, map_location=args.device)
    model.load_state_dict(state_dict, strict=False)
    # 初始化参考模型
    ref_model = MiniMindForCausalLM(lm_config)
    ref_model.load_state_dict(state_dict, strict=False)
    ref_model.eval()
    ref_model.requires_grad_(False)

    Logger(f'LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    ref_model = ref_model.to(args.device)

    return model, ref_model, tokenizer


In [4]:
def dpo_loss(ref_probs, probs, mask, beta):
    # ref_probs 和 probs 都是 shape: (batch_size, seq_len)
    seq_lengths = mask.sum(dim=1, keepdim=True)  # (batch_size, 1)
    ref_probs = (ref_probs * mask).sum(dim=1) / seq_lengths.squeeze()
    probs = (probs * mask).sum(dim=1) / seq_lengths.squeeze()

    # 将 chosen 和 rejected 数据分开
    batch_size = ref_probs.shape[0]
    chosen_ref_probs = ref_probs[:batch_size // 2]
    reject_ref_probs = ref_probs[batch_size // 2:]
    chosen_probs = probs[:batch_size // 2]
    reject_probs = probs[batch_size // 2:]

    pi_logratios = chosen_probs - reject_probs
    ref_logratios = chosen_ref_probs - reject_ref_probs
    logits = pi_logratios - ref_logratios
    loss = -F.logsigmoid(beta * logits)
    return loss.mean()

In [5]:
class Arguments:
  def __init__(self,
      out_dir = "./out",
      epochs = 1,
      batch_size = 32,
      learning_rate = 1e-8,
      device = "cuda" if torch.cuda.is_available() else "cpu",
      dtype = "bfloat16",
      accumulation_steps = 8,
      grad_clip = 1,
      warmup_iters = 0,
      log_interval = 100,
      save_interval = 100,
      local_rank = -1,
      hidden_size = 512,
      num_hidden_layers = 8,
      max_seq_len = 512,
      data_path = "./dataset/dpo_train.jsonl"
  ):
    self.out_dir = out_dir
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.device = device
    self.dtype = dtype
    self.accumulation_steps = accumulation_steps
    self.grad_clip = grad_clip
    self.warmup_iters = warmup_iters
    self.log_interval = log_interval
    self.save_interval = save_interval
    self.local_rank = local_rank
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.max_seq_len = max_seq_len
    self.data_path = data_path
    self.save_dir = None
    self.tokens_per_iter = self.batch_size * self.max_seq_len

In [6]:
def train_epoch(epoch):
    start_time = time.time()
    for step, batch in enumerate(train_loader):
        x_chosen = batch['x_chosen'].to(args.device)
        x_rejected = batch['x_rejected'].to(args.device)
        y_chosen = batch['y_chosen'].to(args.device)
        y_rejected = batch['y_rejected'].to(args.device)
        mask_chosen = batch['mask_chosen'].to(args.device)
        mask_rejected = batch['mask_rejected'].to(args.device)
        x = torch.cat([x_chosen, x_rejected], dim=0)
        y = torch.cat([y_chosen, y_rejected], dim=0)
        mask = torch.cat([mask_chosen, mask_rejected], dim=0)

        lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch, args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            with torch.no_grad():
                ref_outputs = ref_model(x)
                ref_logits = ref_outputs.logits
            ref_probs = logits_to_probs(ref_logits, y)
            ref_probs = ref_probs * mask
            outputs = model(x)
            logits = outputs.logits
            probs = logits_to_probs(logits, y)
            probs = probs * mask
            loss = dpo_loss(ref_probs, probs, mask, beta=0.1)
            loss = loss / args.accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0:
            spend_time = time.time() - start_time
            Logger(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                    epoch + 1,
                    args.epochs,
                    step,
                    iter_per_epoch,
                    loss.item() * args.accumulation_steps,
                    optimizer.param_groups[-1]['lr'],
                    spend_time))

        if (step + 1) % args.save_interval == 0:
            model.eval()
            ckp = f'./out/rlhf_{lm_config.hidden_size}.pth'

            if isinstance(model, torch.nn.parallel.DistributedDataParallel):
                state_dict = model.module.state_dict()
            else:
                state_dict = model.state_dict()
            state_dict = {k: v.half() for k, v in state_dict.items()}  # 半精度保存
            torch.save(state_dict, ckp)
            model.train()

In [ ]:
if __name__ == "__main__":
    args = Arguments()

    lm_config = MiniMindConfig(hidden_size=args.hidden_size, num_hidden_layers=args.num_hidden_layers)
    tokens_per_iter = args.batch_size * args.max_seq_len
    device_type = "cuda" if "cuda" in args.device else "cpu"
    ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast()
    base_seed = 114514
    torch.manual_seed(base_seed)
    torch.cuda.manual_seed(base_seed)

    model, ref_model, tokenizer = init_model(lm_config)

    train_ds = DPODataset(file_path = args.data_path, tokenizer = tokenizer, max_length=args.max_seq_len)
    train_loader = DataLoader(
        train_ds,
        batch_size=args.batch_size,
        pin_memory=True,
        drop_last=False,
        shuffle=False
    )

    scaler = torch.cuda.amp.GradScaler(enabled=(args.dtype in ['float16', 'bfloat16']))
    optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)

    iter_per_epoch = len(train_loader)
    for epoch in range(args.epochs):
        train_epoch(epoch)

LLM总参数量：25.830 百万
Epoch:[1/2000](0/4657) loss:nan lr:0.000000011000 epoch_Time:2.020221471786499min:
Epoch:[1/2000](100/4657) loss:nan lr:0.000000011000 epoch_Time:88.77718877792358min:
Epoch:[1/2000](200/4657) loss:nan lr:0.000000011000 epoch_Time:169.57017850875854min:
Epoch:[1/2000](300/4657) loss:nan lr:0.000000011000 epoch_Time:251.02667713165283min:
Epoch:[1/2000](400/4657) loss:nan lr:0.000000011000 epoch_Time:333.02969765663147min:
Epoch:[1/2000](500/4657) loss:nan lr:0.000000011000 epoch_Time:413.824444770813min:
Epoch:[1/2000](600/4657) loss:nan lr:0.000000011000 epoch_Time:494.73823714256287min:
Epoch:[1/2000](700/4657) loss:nan lr:0.000000011000 epoch_Time:575.4758398532867min:
Epoch:[1/2000](800/4657) loss:nan lr:0.000000011000 epoch_Time:655.8162634372711min:
Epoch:[1/2000](900/4657) loss:nan lr:0.000000011000 epoch_Time:736.0990598201752min:
